In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from time import strftime, localtime
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
import gensim.corpora as corpora
from pprint import pprint
from wordcloud import WordCloud
import os
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import pickle 
from gensim.models.ldamulticore import LdaMulticore
from gensim.corpora import Dictionary

input_file = r"C:\Users\Fabio\OneDrive\Documents\Studies\Final Project\Social-Media-and-News-Article-Sentiment-Analysis-for-Stock-Market-Autotrading\twitter data\Tweets about the Top Companies from 2015 to 2020\Tweet.csv\Tweet.csv"
input_df  = pd.read_csv(input_file, index_col="tweet_id")

#TCT = pd.read_csv(r"C:\Users\Fabio\OneDrive\Documents\Studies\Final Project\Social-Media-and-News-Article-Sentiment-Analysis-for-Stock-Market-Autotrading\twitter data\Tweets about the Top Companies from 2015 to 2020\Company_Tweet.csv\Company_Tweet.csv", index_col="tweet_id")
#TT  = pd.read_csv(r"C:\Users\Fabio\OneDrive\Documents\Studies\Final Project\Social-Media-and-News-Article-Sentiment-Analysis-for-Stock-Market-Autotrading\twitter data\Tweets about the Top Companies from 2015 to 2020\Tweet.csv\Tweet.csv", index_col="tweet_id")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Fabio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:

def import_and_prepare_twitter_data(target_file, filters_dict, start_datetime, finish_datetime):
    epoch_time  = datetime(1970, 1, 1)
    epoch_start = (start_datetime - epoch_time).total_seconds()
    epoch_end   = (finish_datetime - epoch_time).total_seconds()
    global input_df
    
    #trim according to time window    
    input_df = input_df[input_df["post_date"]>epoch_start]
    input_df = input_df[input_df["post_date"]<epoch_end]
    
    #lower case all text
      
    
    
    return input_df
    #produce

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts, stop_words):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

def remove_stopwords_with_wildcards():
    print("S")

    
def return_subject_keys(string_collections, topic_qty = 10, enforced_topics_dict=None, stock_names_list=None, words_to_remove = None, 
                        return_LDA_model=True, return_png_visualisation=False, return_html_visualisation=False, 
                        alpha=0.1, beta=0.01):
    output = []
    for i in range(len(string_collections)):
        string_collections[i] = re.sub('[,\.!?]', "", string_collections[i])
    data = string_collections
    if return_LDA_model < return_html_visualisation:
        raise ValueError("You must return the LDA visualisation if you return the LDA model")
    
    stop_words = stopwords.words('english')
    stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'http', 'goog', 'owlerus', 'read'])
    if stock_names_list != None:
        stop_words.extend(stock_names_list)
    if words_to_remove != None:
        stop_words.extend(words_to_remove)
    data_words = list(sent_to_words(data))
    data_words = remove_stopwords(data_words, stop_words)
       
    if return_png_visualisation==True:
        long_string = "start"
        for w in data_words:
            long_string = long_string + ',' + ','.join(w)
        wordcloud = WordCloud(background_color="white", max_words=1000, contour_width=3, contour_color='steelblue')
        wordcloud.generate(long_string)
        wordcloud.to_image()
        output = output + [wordcloud]
    else:
        output = output + [None]
    
    if return_LDA_model==True:
        # Create Dictionary
        id2word = corpora.Dictionary(data_words)

        # Create Corpus
        texts = data_words

        # Term Document Frequency
        corpus = [id2word.doc2bow(text) for text in texts]

        #translate the enforced_topics_dict input
        eta = None
        if enforced_topics_dict != None:
            eta = np.zeros(len(id2word))
            offset = 1
            for group_num in range(len(enforced_topics_dict)):
                for word in enforced_topics_dict[group_num]:
                    try: 
                        word_id = id2word.token2id[word]
                        eta[word_id] = group_num + offset
                    except:
                        a=1

        # Build LDA model
        lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                               id2word=id2word,
                                               num_topics=topic_qty,
                                               eta=eta,
                                               alpha = alpha, # controls topic sparsity
                                               #beta = beta, # controls word sparsity
                                               workers=2)
        
        # Print the Keyword in the 10 topics
        #pprint(lda_model.print_topics())
        doc_lda = lda_model[corpus]
        output = output + [lda_model, doc_lda, corpus, id2word]
    else:
        output = output + [None, None, None]
        
        
    if return_html_visualisation==True:
        pyLDAvis.enable_notebook()
        LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
        output = output + [LDAvis_prepared]
    else:
        output = output + [None]
    
    return tuple(output)







def prep_twitter_text_for_subject_discovery(body_list, df_stocks_list_file, eliminate_company_names_and_tickers=True):
    stock_name_stopwords    = ["limited", "lmt", "ltd", "incorporated", "inc.", "inc", "froup", "gmbh", "fund", ".com", "corp", "trust", "company", "global", ".", "!", "the"]
    stocks_list             = []
    tickers_list            = []
    other_stopwords         = ["tesla", "TSLA", "elon", "musk", "baird", "robert", "pm", "androidwear", "android", "robert", "ab", "ae", "dlvrit", "https", "iphone", "inc", "new", "dlvrit", "py", "twitter", "cityfalconcom", "aapl", "ing", "ios", "samsung", "ipad", "phones", "cityfalconcom", "us", "bitly", "utmmpaign", "etexclusivecom", "cityfalcon", "owler", "com", "stock", "stocks", "buy", "bitly", "dlvrit", "alexa", "zprio", "billion", "seekalphacom", "et", "alphet", "seekalpha", "googl", "zprio", "trad", "jt", "windows", "adw", "ifttt", "ihadvfn", "nmona", "pphppid", "st", "bza", "twits", "biness", "tim", "ba", "se", "rat", "article"]

    ticker_to_name_dict     = dict()
    body_list               = list(body_list.map(lambda x: x.lower()).values)


    for stock_id in df_stocks_list_file.axes[0]:
        ticker_to_name_dict[df_stocks_list_file["Ticker"][stock_id]] = df_stocks_list_file["Name"][stock_id]
    
    if eliminate_company_names_and_tickers == True:
        stocks_list  = list(df_stocks_list_file["Name"].map(lambda x: x.lower()).values)
        tickers_list_1 = list(df_stocks_list_file["Ticker"].map(lambda x: "$" + x.lower()).values)
        tickers_list_2 = list(df_stocks_list_file["Ticker"].map(lambda x: " " + x.lower()).values)
        tickers_list_3 = list(df_stocks_list_file["Ticker"].map(lambda x: x.lower()).values + " ")
        tickers_list.reverse()
        stocks_list_extended = []
        for stock_name_ in stocks_list:
            edited = False
            stock_name = stock_name_
            for stopwords in stock_name_stopwords:
                if stopwords in stock_name:
                    edited = True
                    stock_name = stock_name.replace(stopwords, "").strip()
            if edited == True:
                stock_name = stock_name.replace("  "," ")
                stock_name = stock_name.replace("  "," ")
                stocks_list_extended = stocks_list_extended + [stock_name]
    
    #order lists
    stocks_list.sort(reverse=True)
    tickers_list_1.sort(reverse=True)
    tickers_list_2.sort(reverse=True)
    stocks_list_extended.sort(reverse=True)
    
    
    for i, body in zip(range(len(body_list)), body_list):
        for x in stocks_list + tickers_list_1 + tickers_list_2 + tickers_list_3 + stocks_list_extended + other_stopwords:
            if " " + x + " " in body_list[i]:
                body_list[i] = body_list[i].replace(x,"")
    
    
    #for x in stocks_list + tickers_list + stocks_list_extended:
    #    body_list = map(lambda x: x.replace(x, ""), body_list)
        
    #body_list = list(body_list)
    
    return body_list

def save_load_subject_keys(output_folder_name, run_name, wordcloud=None, doc_lda=None, corpus=None, id2word=None, visualisation=None):
    #run_name = "test"
    if not os.path.isdir(output_folder_name):
    #os.path.dirname(os.path.dirname(path))
        os.mkdir(output_folder_name)
    
    if wordcloud != None:
        wordcloud.to_file(output_folder_name + "word cloud_" + run_name + ".png")

    #if doc_lda != None or corpus != None or id2word != None:
    #    
    #    try:
    #        LDAvis_prepared = gensimvis.prepare(doc_lda, corpus, id2word)
    #        with open(LDAvis_data_filepath, 'wb') as f:
    #            pickle.dump(LDAvis_prepared, f)
    #    except:
    #        print("error, outputs doc_lda, corpus & id2word are not yet programmed to save")
        
    if visualisation != None:
        pyLDAvis.save_html(visualisation, output_folder_name + "word cloud_" + run_name +'.html')


def export_words_within_topics_count(lda_model):
    strings_dict = dict()
    for topic in lda_model.print_topics():
        tuples = topic[1].split("+")
        for tup in tuples:
            word = tup.split("+")[0].split("*")[1]
            word = re.sub('[^a-zA-Z]', '', word)
            num  = tup.split("+")[0].split("*")[0]
            if not word in strings_dict.keys():
                strings_dict[word] = 0
            strings_dict[word] += float(num)

    words_within_topics_count = pd.DataFrame(list(strings_dict.items()), columns=['keys', 'values'])
    words_within_topics_count.sort_values(by="values", ascending=False, inplace=True)
    model_time = datetime.now()
    model_time = model_time.strftime("%Y%m%d_%H%M")
    words_within_topics_count.to_csv("C:\\Users\\Fabio\\OneDrive\\Documents\\Studies\\Final Project\\Social-Media-and-News-Article-Sentiment-Analysis-for-Stock-Market-Autotrading\\test books\\output\\word_topics" + model_time + ".csv")


#parameters
tweet_qty = int(1e2)
topic_qty = 5
alpha=0.2
beta=0.02



input_file = r"C:\Users\Fabio\OneDrive\Documents\Studies\Final Project\Social-Media-and-News-Article-Sentiment-Analysis-for-Stock-Market-Autotrading\twitter data\Tweets about the Top Companies from 2015 to 2020\Tweet.csv\Tweet.csv"
start_datetime = datetime.strptime('01/01/17 00:00:00', '%m/%d/%y %H:%M:%S')
end_datetime = datetime.strptime('01/01/18 00:00:00', '%m/%d/%y %H:%M:%S')
enforced_topics_dict = [
    ['investment', 'financing', 'losses'],
    ['risk', 'exposure', 'liability'],
    ["financial forces" , "growth", "interest rates"]]
df_stocks_list_file     = pd.read_csv(r"C:\Users\Fabio\OneDrive\Documents\Studies\Final Project\Social-Media-and-News-Article-Sentiment-Analysis-for-Stock-Market-Autotrading\data\stock_info.csv")
output_folder_name = "C:\\Users\\Fabio\\OneDrive\\Documents\\Studies\\Final Project\\Social-Media-and-News-Article-Sentiment-Analysis-for-Stock-Market-Autotrading\\test books\\output\\"


#data prep
print(datetime.now().strftime("%H:%M:%S"))
df_prepped_sentiment                                = import_and_prepare_twitter_data(input_file, dict(), start_datetime, end_datetime)
print("--------------------------------1--------------------------------")
print(datetime.now().strftime("%H:%M:%S"))
df_prepped_sentiment_company_agnostic               = prep_twitter_text_for_subject_discovery(df_prepped_sentiment["body"][:tweet_qty], df_stocks_list_file, eliminate_company_names_and_tickers=True)
print("--------------------------------2--------------------------------")
print(datetime.now().strftime("%H:%M:%S"))
wordcloud, lda_model, doc_lda, corpus, id2word, visualisation  = return_subject_keys(df_prepped_sentiment_company_agnostic, topic_qty = topic_qty, alpha=alpha, beta=beta,
                                                                                     enforced_topics_dict=enforced_topics_dict, return_LDA_model=True, return_png_visualisation=True, return_html_visualisation=True)
print("--------------------------------3--------------------------------")
print(datetime.now().strftime("%H:%M:%S"))
save_load_subject_keys(output_folder_name, str(tweet_qty), wordcloud=wordcloud, doc_lda=doc_lda, corpus=corpus, id2word=id2word, visualisation=visualisation)
export_words_within_topics_count(lda_model)
print("--------------------------------4--------------------------------")
print(datetime.now().strftime("%H:%M:%S"))



<>:36: DeprecationWarning: invalid escape sequence '\.'
<>:36: DeprecationWarning: invalid escape sequence '\.'
C:\Users\Fabio\AppData\Local\Temp\ipykernel_12316\3240728472.py:36: DeprecationWarning: invalid escape sequence '\.'
  string_collections[i] = re.sub('[,\.!?]', "", string_collections[i])


22:07:44
--------------------------------1--------------------------------
22:07:44
--------------------------------2--------------------------------
22:07:46


c:\Users\Fabio\AppData\Local\Programs\Python\Python311\Lib\site-packages\wordcloud\wordcloud.py:106: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  self.colormap = plt.cm.get_cmap(colormap)


--------------------------------3--------------------------------
22:07:49
--------------------------------4--------------------------------
22:07:49


In [7]:
def prep_twitter_text_for_subject_discovery(body_list, df_stocks_list_file, eliminate_company_names_and_tickers=True):

    
    stock_name_stopwords    = ["limited", "lmt", "ltd", "incorporated", "inc.", "inc", "froup", "gmbh", "fund", ".com", "corp", "trust", "company", "global", ".", "!", "the"]
    stocks_list             = []
    tickers_list            = []
    other_stopwords         = ["tesla", "TSLA", "elon", "musk", "baird", "robert", "pm", "androidwear", "android", "robert", "ab", "ae", "dlvrit", "https", "iphone", "inc", "new", "dlvrit", "py", "twitter", "cityfalconcom", "aapl", "ing", "ios", "samsung", "ipad", "phones", "cityfalconcom", "us", "bitly", "utmmpaign", "etexclusivecom", "cityfalcon", "owler", "com", "stock", "stocks", "buy", "bitly", "dlvrit", "alexa", "zprio", "billion", "seekalphacom", "et", "alphet", "seekalpha", "googl", "zprio", "trad", "jt", "windows", "adw", "ifttt", "ihadvfn", "nmona", "pphppid", "st", "bza", "twits", "biness", "tim", "ba", "se", "rat", "article"]

    ticker_to_name_dict     = dict()
    body_list               = list(body_list.map(lambda x: x.lower()).values)


    for stock_id in df_stocks_list_file.axes[0]:
        ticker_to_name_dict[df_stocks_list_file["Ticker"][stock_id]] = df_stocks_list_file["Name"][stock_id]
    
    if eliminate_company_names_and_tickers == True:
        stocks_list  = list(df_stocks_list_file["Name"].map(lambda x: x.lower()).values)
        tickers_list_1 = list(df_stocks_list_file["Ticker"].map(lambda x: "$" + x.lower()).values)
        tickers_list_2 = list(df_stocks_list_file["Ticker"].map(lambda x: " " + x.lower()).values)
        tickers_list_3 = list(df_stocks_list_file["Ticker"].map(lambda x: x.lower()).values + " ")
        tickers_list.reverse()
        stocks_list_extended = []
        for stock_name_ in stocks_list:
            edited = False
            stock_name = stock_name_
            for stopwords in stock_name_stopwords:
                if stopwords in stock_name:
                    edited = True
                    stock_name = stock_name.replace(stopwords, "").strip()
            if edited == True:
                stock_name = stock_name.replace("  "," ")
                stock_name = stock_name.replace("  "," ")
                stocks_list_extended = stocks_list_extended + [stock_name]
    
    #order lists
    stocks_list.sort(reverse=True)
    tickers_list_1.sort(reverse=True)
    tickers_list_2.sort(reverse=True)
    stocks_list_extended.sort(reverse=True)
    
    
    for i, body in zip(range(len(body_list)), body_list):
        for x in stocks_list + tickers_list_1 + tickers_list_2 + tickers_list_3 + stocks_list_extended + other_stopwords:
            if " " + x + " " in body_list[i]:
                body_list[i] = body_list[i].replace(x,"")
    
    return body_list


df_prepped_sentiment_company_agnostic = prep_twitter_text_for_subject_discovery(df_prepped_sentiment["body"][:tweet_qty], df_stocks_list_file, eliminate_company_names_and_tickers=True)

for i in range(50):
    print(df_prepped_sentiment_company_agnostic[i])



how to improve your #trading using these 3  factors   $terp   #options #stockstotrade $uso https://youtube.com/watch?v=julzm-4gsd0&btz49=0000010601…
see why these assets are trending in 1 watchlist #fintech  $blockchain http://cityfalcon.com/watchlists?assets=%23fintech,,$blockchain…
review: bragi's headphone offers a solid alternative to  airpods http://dlvr.it/n0sxd9 #insider $aapl
$spx $spy $compq  $rut $djia $dia $vix $uso  $study weekend/year end analysis http://letsinvestnow.com
it feels like a secret, but there is an answer:$spy  $xlf    $sbuxwelcome to the worldhttp://news.cmlviz.com/2016/12/29/welcome-to-the-world-of-back-testing-again.html…
it does   :)
these assets are seeing a jump in tweets #fintech  $blockchain http://cityfalcon.com/watchlists?assets=%23fintech,,$blockchain…
$tsla"i continue to believe that it’s the market’s biggest single-company  bubble."mark spiegel
$data can't find a bottom.  maybe next year but watch  bi data visualization
$adp percent change updated

In [15]:
df_stocks_list_file     = pd.read_csv(r"C:\Users\Fabio\OneDrive\Documents\Studies\Final Project\Social-Media-and-News-Article-Sentiment-Analysis-for-Stock-Market-Autotrading\data\stock_info.csv")

In [14]:
def new_method(input_list, df_stocks_list_file=df_stocks_list_file):
    #prep parameters
    stocks_list  = list(df_stocks_list_file["Name"].map(lambda x: x.lower()).values)
    tickers_list = list(df_stocks_list_file["Ticker"].map(lambda x: x.lower()).values)


    #prep variables
    split_tweets = []
    for tweet in input_list:
        split_tweet_pre = tweet.split(" ")
        split_tweet = []
        for word in split_tweet_pre:
            #split_tweet = split_tweet + [" " + word.lower() + " "]
            split_tweet = split_tweet + [word.lower()]
        split_tweets = split_tweets + [split_tweet]

    
    # remove words containing "x"
    death_characters = ["$", "apple", "tesla", "http", "@"]
    for tweet in split_tweets:
        for char in death_characters:
            for word in reversed(tweet):
                if char in word:
                    tweet.remove(word)
    
    # remove words equalling "x"
    
    
    
    
    
    
    #finalise    
    output = []
    for split_tweet in split_tweets:
        a = list(map(lambda x: x.strip(), split_tweet))
        output = output + [" ".join(a).replace("  "," ")]
    
    
    return input_list


new_filter = new_method(list(input_df["body"][:1000:10]), df_stocks_list_file=df_stocks_list_file)

for i in range(10):
    print(new_filter[i])



hello
lx21 made $10,008  on $AAPL -Check it out! http://profit.ly/1MnD8s?aff=202 Learn #howtotrade http://bit.ly/1c1NljX $EXE $WATT $IMRS $CACH $GMO
Swing Trading: Up To 8.91% Return In 14 Days http://ow.ly/GDkrT #swingtrading #forecast #techstock $MWW $AAPL $TSLA
#GOOGLE 'C' : Chinese paper blames Google over Gmail blocking  http://4-traders.com/GOOGLE-INC-C-16118013/news/GOOGLE-C--Chinese-paper-blames-Google-over-Gmail-blocking-19608660/… $GOOG
http://StockAviator.com....Top penny stocks, NYSE, and NASDAQ trades from the stock market today. +296% last week.   $KGC $ERBB $F $MSFT
$AMZN OI for maturity 01/02/2015. 275.00 Highest put. 320.00 Highest call. http://maximum-pain.com/open-interest.aspx?s=AMZN&e=01/02/2015…
Can't update my iPad because of this... Apple sued for shrinking storage space on 16GB devices thanks to iOS 8 $AAPL
World’s 400 Richest People Add $92-B In 2014: World’s 400 Richest People Add $92-B In 2014 $AMZN, $BABA, $BX, ... http://tinyurl.com/ne9rgyc
World’s 400 Ric

In [21]:
print("h")

h


In [11]:
# Define your list of tuples
expressions = a

# Define a set to store the variable names
var_names = set()

# Loop through each tuple and extract the variable names
for expr in expressions:
    var_names.update(expr[1].split(' + '))
    var_names.update(expr[1].split(' - '))

# Convert the set to a list and sort alphabetically
var_names = sorted(list(var_names))

chars_to_remove = '0123456789"*'
translator = str.maketrans('', '', chars_to_remove)
var_names = [text.translate(translator) for text in var_names]


# Define a dictionary to store the total coefficients for each expression
coeff_totals = {}

# Loop through each tuple and calculate the total coefficient for each variable
for expr in expressions:
    coeff_total = 0
    for var in var_names:
        if var in expr[1]:
            if '+' in expr[1]:
                coeff_total += float(expr[1].split(var + ' + ')[0])
            elif '-' in expr[1]:
                coeff_total -= float(expr[1].split(var + ' - ')[0])
            else:
                coeff_total += float(expr[1].split(var)[0])
    coeff_totals[expr[1]] = coeff_total

# Sort the dictionary by the total coefficient values
sorted_coeffs = sorted(coeff_totals.items(), key=lambda x: x[1], reverse=True)

# Print the variable names and the sorted list of tuples
print('Variable names: ', var_names)
print('Expressions sorted by coefficient total:')
for expr in sorted_coeffs:
    print(expr[1])


Variable names:  ['.ae', '.android', '.doj', '.eu', '.ftc', '.great', '.hstdyr', '.slideshow', '.sor', '.valley', '.aepay', '.degree', '.dropped', '.fitbit', '.iovates', '.smelted', '.stock', '.tablet', '.tracking', '.using', '.vat', '.video', '.weekly', '.days', '.disdained', '.istmt', '.reality', '.turns', '.ultrabook', '.accounting', '.companies', '.feel', '.https', '.saturday', '.sm', '.vincepaver', '.year', '.news', '.december', '.dlvrit', '.highest', '.may', '.pm', '.saturday', '.think', '.updated', '.world', '.ae', '.ai', '.buy', '.companies', '.growth', '.new', '.growth', '.growth', '.com', '.decline', '.fortifies', '.perct', '.record', '.repurchases', '.retail', '.share', '.tweet', '.utm_mpaign', '.news', '.androidwear', '.growth', '.growth + .highest + .may + .pm + .think + .december + .updated + .saturday + .dlvrit + .news', '.looks', '.wearables', '.blockchain', '.december', '.december + .pm + .dlvrit + .saturday + .blockchain + .fintech + .updated + .growth + .new + .ae', 